In [1]:
import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  #忽略烦人的警告
tf.logging.set_verbosity(tf.logging.ERROR)   #忽略烦人的警告
print(tf.__version__)
import numpy as np
from sklearn.decomposition import PCA

import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
import pickle   #对象存储用

from sklearn import preprocessing

dataroot_1 = "./UNSW-NB15 - CSV Files/a part of training and testing set/UNSW_NB15_training-set.csv"
dataroot_2 = "./UNSW-NB15 - CSV Files/a part of training and testing set/UNSW_NB15_testing-set.csv"

1.13.1


In [2]:
df_train_1 = pd.read_csv(dataroot_1,header=0,low_memory=False)  
df_train_2 = pd.read_csv(dataroot_2,header=0,low_memory=False)  
train_dataset = df_train_1.append([df_train_2])

In [3]:
#选择需要进行创建字典的特征。实际有23个特征加1个标签
def feature_selection(dataset):
    return dataset[[
        'dur', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl', 'dttl', 'sload', 
        'dload', 'sloss', 'sinpkt', 'sjit', 'tcprtt', 'synack', 'ackdat', 
        'smean', 'dmean', 'ct_srv_src', 'ct_state_ttl', 'ct_src_dport_ltm', 
        'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'ct_srv_dst', 'attack_cat'
    ]]

def filter_useless_feature(dataset):
    dataset = dataset.drop(['label'], axis=1)    #两个特征没用
    dataset = dataset.drop(['id'], axis=1)
    return dataset

def filter_labels(dataset):
    dataset['attack_cat'] = dataset['attack_cat'].apply(labels_map)
    return dataset

def labels_map(label):
    if(type(label) == int):      #已是数字，无需处理
        return label
    
    if label == 'Normal':
        return 0
    if label == 'Fuzzers':
        return 1
    if label == 'Analysis':
        return 2
    if label == 'Backdoor':
        return 3
    if label == 'DoS':
        return 4
    if label == 'Exploits':
        return 5
    if label == 'Generic':
        return 6
    if label == 'Reconnaissance':
        return 7
    if label == 'Shellcode':
        return 8
    if label == 'Worms':
        return 9
    return -1


In [4]:
def save_obj(obj, name ):
    with open(''+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

#数理统计方法，按5%区间进行数理统计
def statistics(dataset, feature_index):
    intervals = {'{0}-{1}'.format(5 * x, 5 * (x+1) ): 0 for x in range(20)}
    for _ in dataset[:,feature_index]:
        for interval in intervals:
            start, end = tuple(interval.split('-'))
            if int(start) <= _*100 <= int(end):
                intervals[interval] += 1
    return intervals

In [5]:
#特征选择
train_dataset = feature_selection(train_dataset)
train_dataset = filter_labels(train_dataset)

In [6]:
print(train_dataset.shape)
print(train_dataset.columns)
print(train_dataset[0:1])
feature_num = train_dataset.shape[1]
feature_num = feature_num -1 #去掉最后一个标签
print(feature_num)

(257673, 24)
Index(['dur', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl', 'dttl', 'sload',
       'dload', 'sloss', 'sinpkt', 'sjit', 'tcprtt', 'synack', 'ackdat',
       'smean', 'dmean', 'ct_srv_src', 'ct_state_ttl', 'ct_src_dport_ltm',
       'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'ct_srv_dst', 'attack_cat'],
      dtype='object')
        dur  dpkts  sbytes  dbytes        rate  sttl  dttl        sload  \
0  0.000011      0     496       0  90909.0902   254     0  180363632.0   

   dload  sloss  ...  ackdat  smean  dmean  ct_srv_src  ct_state_ttl  \
0    0.0      0  ...     0.0    248      0           2             2   

   ct_src_dport_ltm  ct_dst_sport_ltm  ct_dst_src_ltm  ct_srv_dst  attack_cat  
0                 1                 1               2           2           0  

[1 rows x 24 columns]
23


In [7]:
##收集normal的distribution dictionary
print("Collecting normal distribution dictionary")

#Data preprocessing
normal_df = train_dataset.loc[train_dataset['attack_cat'] == 0]
#Feature scaling
scaler = preprocessing.MinMaxScaler()
normalMatrix=scaler.fit_transform(normal_df.values)
print(normalMatrix.shape)

#normal的分布统计字典创建并存储至文件
normal_Distribution_set = {}
for i in range(feature_num):
    StatisticObj = statistics(normalMatrix, i)
    normal_Distribution_set[str(i)] = StatisticObj

print(normal_Distribution_set)
save_obj(normal_Distribution_set, "normal_Distribution_set")

print("End of collecting normal distribution dictionary")

(93000, 24)
{'0': {'0-5': 89133, '5-10': 1271, '10-15': 684, '15-20': 435, '20-25': 327, '25-30': 188, '30-35': 93, '35-40': 62, '40-45': 54, '45-50': 42, '50-55': 130, '55-60': 40, '60-65': 45, '65-70': 58, '70-75': 31, '75-80': 31, '80-85': 131, '85-90': 21, '90-95': 10, '95-100': 214}, '1': {'0-5': 87534, '5-10': 3338, '10-15': 101, '15-20': 8, '20-25': 224, '25-30': 669, '30-35': 48, '35-40': 44, '40-45': 625, '45-50': 371, '50-55': 346, '55-60': 5, '60-65': 2, '65-70': 3, '70-75': 25, '75-80': 0, '80-85': 2, '85-90': 0, '90-95': 0, '95-100': 1}, '2': {'0-5': 87808, '5-10': 1860, '10-15': 2468, '15-20': 5, '20-25': 589, '25-30': 3, '30-35': 1, '35-40': 107, '40-45': 81, '45-50': 2, '50-55': 3, '55-60': 1, '60-65': 7, '65-70': 11, '70-75': 12, '75-80': 10, '80-85': 15, '85-90': 0, '90-95': 0, '95-100': 17}, '3': {'0-5': 90743, '5-10': 288, '10-15': 2, '15-20': 9, '20-25': 809, '25-30': 50, '30-35': 50, '35-40': 9, '40-45': 42, '45-50': 960, '50-55': 0, '55-60': 5, '60-65': 2, '65-70

In [8]:
##收集Fuzzers的distribution dictionary
print("Collecting Fuzzers distribution dictionary")

#Data preprocessing
Fuzzers_df = train_dataset.loc[train_dataset['attack_cat'] == 1]
#Feature scaling
FuzzersMatrix=scaler.fit_transform(Fuzzers_df.values)
print(FuzzersMatrix.shape)

#normal的分布统计字典创建并存储至文件
Fuzzers_Distribution_set = {}
for i in range(feature_num):
    StatisticObj = statistics(FuzzersMatrix, i)
    Fuzzers_Distribution_set[str(i)] = StatisticObj

print(Fuzzers_Distribution_set)
save_obj(Fuzzers_Distribution_set, "Fuzzers_Distribution_set")

print("End of collecting Fuzzers distribution dictionary")

(24246, 24)
{'0': {'0-5': 22728, '5-10': 270, '10-15': 70, '15-20': 36, '20-25': 31, '25-30': 21, '30-35': 34, '35-40': 26, '40-45': 15, '45-50': 12, '50-55': 231, '55-60': 78, '60-65': 46, '65-70': 60, '70-75': 76, '75-80': 36, '80-85': 42, '85-90': 75, '90-95': 75, '95-100': 284}, '1': {'0-5': 23952, '5-10': 251, '10-15': 10, '15-20': 0, '20-25': 0, '25-30': 20, '30-35': 5, '35-40': 0, '40-45': 0, '45-50': 0, '50-55': 0, '55-60': 7, '60-65': 0, '65-70': 0, '70-75': 0, '75-80': 0, '80-85': 0, '85-90': 0, '90-95': 0, '95-100': 1}, '2': {'0-5': 23983, '5-10': 203, '10-15': 45, '15-20': 8, '20-25': 0, '25-30': 0, '30-35': 0, '35-40': 0, '40-45': 0, '45-50': 0, '50-55': 0, '55-60': 0, '60-65': 0, '65-70': 0, '70-75': 0, '75-80': 0, '80-85': 0, '85-90': 0, '90-95': 0, '95-100': 7}, '3': {'0-5': 24219, '5-10': 0, '10-15': 1, '15-20': 0, '20-25': 25, '25-30': 0, '30-35': 0, '35-40': 0, '40-45': 0, '45-50': 0, '50-55': 0, '55-60': 0, '60-65': 0, '65-70': 0, '70-75': 0, '75-80': 0, '80-85': 0,

In [9]:
##收集Analysis的distribution dictionary
print("Collecting Analysis distribution dictionary")

#Data preprocessing
Analysis_df = train_dataset.loc[train_dataset['attack_cat'] == 2]
#Feature scaling
AnalysisMatrix=scaler.fit_transform(Analysis_df.values)
print(AnalysisMatrix.shape)

#normal的分布统计字典创建并存储至文件
Analysis_Distribution_set = {}
for i in range(feature_num):
    StatisticObj = statistics(AnalysisMatrix, i)
    Analysis_Distribution_set[str(i)] = StatisticObj

print(Fuzzers_Distribution_set)
save_obj(Analysis_Distribution_set, "Analysis_Distribution_set")

print("End of collecting Analysis distribution dictionary")

(2677, 24)
{'0': {'0-5': 22728, '5-10': 270, '10-15': 70, '15-20': 36, '20-25': 31, '25-30': 21, '30-35': 34, '35-40': 26, '40-45': 15, '45-50': 12, '50-55': 231, '55-60': 78, '60-65': 46, '65-70': 60, '70-75': 76, '75-80': 36, '80-85': 42, '85-90': 75, '90-95': 75, '95-100': 284}, '1': {'0-5': 23952, '5-10': 251, '10-15': 10, '15-20': 0, '20-25': 0, '25-30': 20, '30-35': 5, '35-40': 0, '40-45': 0, '45-50': 0, '50-55': 0, '55-60': 7, '60-65': 0, '65-70': 0, '70-75': 0, '75-80': 0, '80-85': 0, '85-90': 0, '90-95': 0, '95-100': 1}, '2': {'0-5': 23983, '5-10': 203, '10-15': 45, '15-20': 8, '20-25': 0, '25-30': 0, '30-35': 0, '35-40': 0, '40-45': 0, '45-50': 0, '50-55': 0, '55-60': 0, '60-65': 0, '65-70': 0, '70-75': 0, '75-80': 0, '80-85': 0, '85-90': 0, '90-95': 0, '95-100': 7}, '3': {'0-5': 24219, '5-10': 0, '10-15': 1, '15-20': 0, '20-25': 25, '25-30': 0, '30-35': 0, '35-40': 0, '40-45': 0, '45-50': 0, '50-55': 0, '55-60': 0, '60-65': 0, '65-70': 0, '70-75': 0, '75-80': 0, '80-85': 0, 

In [10]:
##收集Backdoor的distribution dictionary
print("Collecting Backdoor distribution dictionary")

#Data preprocessing
Backdoor_df = train_dataset.loc[train_dataset['attack_cat'] == 3]
#Feature scaling
BackdoorMatrix=scaler.fit_transform(Backdoor_df.values)
print(BackdoorMatrix.shape)

#normal的分布统计字典创建并存储至文件
Backdoor_Distribution_set = {}
for i in range(feature_num):
    StatisticObj = statistics(BackdoorMatrix, i)
    Backdoor_Distribution_set[str(i)] = StatisticObj

print(Backdoor_Distribution_set)
save_obj(Backdoor_Distribution_set, "Backdoor_Distribution_set")

print("End of collecting Backdoor distribution dictionary")

(2329, 24)
{'0': {'0-5': 2177, '5-10': 18, '10-15': 16, '15-20': 9, '20-25': 7, '25-30': 7, '30-35': 4, '35-40': 6, '40-45': 5, '45-50': 4, '50-55': 8, '55-60': 3, '60-65': 5, '65-70': 1, '70-75': 4, '75-80': 4, '80-85': 0, '85-90': 2, '90-95': 4, '95-100': 45}, '1': {'0-5': 1998, '5-10': 2, '10-15': 58, '15-20': 184, '20-25': 55, '25-30': 21, '30-35': 4, '35-40': 2, '40-45': 0, '45-50': 6, '50-55': 10, '55-60': 3, '60-65': 0, '65-70': 0, '70-75': 0, '75-80': 0, '80-85': 0, '85-90': 0, '90-95': 0, '95-100': 8}, '2': {'0-5': 2279, '5-10': 33, '10-15': 8, '15-20': 1, '20-25': 2, '25-30': 0, '30-35': 0, '35-40': 0, '40-45': 0, '45-50': 0, '50-55': 0, '55-60': 0, '60-65': 0, '65-70': 0, '70-75': 0, '75-80': 0, '80-85': 0, '85-90': 0, '90-95': 0, '95-100': 6}, '3': {'0-5': 2301, '5-10': 5, '10-15': 0, '15-20': 7, '20-25': 0, '25-30': 2, '30-35': 6, '35-40': 0, '40-45': 0, '45-50': 0, '50-55': 0, '55-60': 0, '60-65': 0, '65-70': 0, '70-75': 0, '75-80': 0, '80-85': 0, '85-90': 0, '90-95': 0, 

In [11]:
##收集DOS的distribution dictionary
print("Collecting DOS distribution dictionary")

#Data preprocessing
DOS_df = train_dataset.loc[train_dataset['attack_cat'] == 4]
#Feature scaling
DOSMatrix=scaler.fit_transform(DOS_df.values)
print(DOSMatrix.shape)

#normal的分布统计字典创建并存储至文件
DOS_Distribution_set = {}
for i in range(feature_num):
    StatisticObj = statistics(DOSMatrix, i)
    DOS_Distribution_set[str(i)] = StatisticObj

print(DOS_Distribution_set)
save_obj(DOS_Distribution_set, "DOS_Distribution_set")

print("End of collecting DOS distribution dictionary")

(16353, 24)
{'0': {'0-5': 15167, '5-10': 205, '10-15': 110, '15-20': 92, '20-25': 50, '25-30': 44, '30-35': 14, '35-40': 38, '40-45': 25, '45-50': 33, '50-55': 44, '55-60': 21, '60-65': 35, '65-70': 14, '70-75': 25, '75-80': 28, '80-85': 1, '85-90': 11, '90-95': 31, '95-100': 365}, '1': {'0-5': 16277, '5-10': 27, '10-15': 12, '15-20': 1, '20-25': 3, '25-30': 1, '30-35': 4, '35-40': 1, '40-45': 8, '45-50': 4, '50-55': 1, '55-60': 2, '60-65': 3, '65-70': 0, '70-75': 2, '75-80': 0, '80-85': 1, '85-90': 1, '90-95': 0, '95-100': 5}, '2': {'0-5': 16296, '5-10': 28, '10-15': 0, '15-20': 3, '20-25': 3, '25-30': 2, '30-35': 0, '35-40': 0, '40-45': 1, '45-50': 2, '50-55': 4, '55-60': 1, '60-65': 0, '65-70': 1, '70-75': 2, '75-80': 3, '80-85': 2, '85-90': 2, '90-95': 1, '95-100': 2}, '3': {'0-5': 16298, '5-10': 13, '10-15': 4, '15-20': 1, '20-25': 2, '25-30': 3, '30-35': 4, '35-40': 0, '40-45': 9, '45-50': 4, '50-55': 1, '55-60': 2, '60-65': 2, '65-70': 1, '70-75': 2, '75-80': 0, '80-85': 1, '85-

In [12]:
##收集Exploits的distribution dictionary
print("Collecting Exploits distribution dictionary")

#Data preprocessing
Exploits_df = train_dataset.loc[train_dataset['attack_cat'] == 5]
#Feature scaling
ExploitsMatrix=scaler.fit_transform(Exploits_df.values)
print(ExploitsMatrix.shape)

#normal的分布统计字典创建并存储至文件
Exploits_Distribution_set = {}
for i in range(feature_num):
    StatisticObj = statistics(ExploitsMatrix, i)
    Exploits_Distribution_set[str(i)] = StatisticObj

print(Exploits_Distribution_set)
save_obj(Exploits_Distribution_set, "Exploits_Distribution_set")

print("End of collecting Exploits distribution dictionary")

(44525, 24)
{'0': {'0-5': 39756, '5-10': 2628, '10-15': 608, '15-20': 222, '20-25': 140, '25-30': 103, '30-35': 38, '35-40': 86, '40-45': 50, '45-50': 67, '50-55': 74, '55-60': 19, '60-65': 59, '65-70': 19, '70-75': 47, '75-80': 60, '80-85': 8, '85-90': 24, '90-95': 46, '95-100': 471}, '1': {'0-5': 43633, '5-10': 499, '10-15': 121, '15-20': 92, '20-25': 69, '25-30': 17, '30-35': 22, '35-40': 20, '40-45': 14, '45-50': 6, '50-55': 6, '55-60': 3, '60-65': 2, '65-70': 1, '70-75': 7, '75-80': 3, '80-85': 2, '85-90': 2, '90-95': 2, '95-100': 4}, '2': {'0-5': 44271, '5-10': 111, '10-15': 17, '15-20': 19, '20-25': 6, '25-30': 12, '30-35': 4, '35-40': 14, '40-45': 11, '45-50': 16, '50-55': 18, '55-60': 3, '60-65': 8, '65-70': 3, '70-75': 3, '75-80': 1, '80-85': 2, '85-90': 3, '90-95': 1, '95-100': 2}, '3': {'0-5': 43809, '5-10': 452, '10-15': 107, '15-20': 46, '20-25': 24, '25-30': 5, '30-35': 19, '35-40': 14, '40-45': 11, '45-50': 6, '50-55': 6, '55-60': 3, '60-65': 2, '65-70': 0, '70-75': 8, 

In [13]:
##收集Generic的distribution dictionary
print("Collecting Generic distribution dictionary")

#Data preprocessing
Generic_df = train_dataset.loc[train_dataset['attack_cat'] == 6]
#Feature scaling
GenericMatrix=scaler.fit_transform(Generic_df.values)
print(GenericMatrix.shape)

#normal的分布统计字典创建并存储至文件
Generic_Distribution_set = {}
for i in range(feature_num):
    StatisticObj = statistics(GenericMatrix, i)
    Generic_Distribution_set[str(i)] = StatisticObj

print(Generic_Distribution_set)
save_obj(Generic_Distribution_set, "Generic_Distribution_set")

print("End of collecting Generic distribution dictionary")

(58871, 24)
{'0': {'0-5': 58705, '5-10': 71, '10-15': 23, '15-20': 11, '20-25': 4, '25-30': 6, '30-35': 8, '35-40': 8, '40-45': 1, '45-50': 2, '50-55': 4, '55-60': 1, '60-65': 3, '65-70': 3, '70-75': 3, '75-80': 2, '80-85': 0, '85-90': 1, '90-95': 0, '95-100': 15}, '1': {'0-5': 58830, '5-10': 24, '10-15': 3, '15-20': 3, '20-25': 4, '25-30': 2, '30-35': 2, '35-40': 0, '40-45': 0, '45-50': 0, '50-55': 2, '55-60': 0, '60-65': 0, '65-70': 0, '70-75': 0, '75-80': 0, '80-85': 0, '85-90': 0, '90-95': 0, '95-100': 1}, '2': {'0-5': 58858, '5-10': 8, '10-15': 0, '15-20': 2, '20-25': 0, '25-30': 0, '30-35': 0, '35-40': 0, '40-45': 0, '45-50': 0, '50-55': 0, '55-60': 0, '60-65': 0, '65-70': 0, '70-75': 0, '75-80': 0, '80-85': 0, '85-90': 0, '90-95': 0, '95-100': 3}, '3': {'0-5': 58833, '5-10': 21, '10-15': 3, '15-20': 3, '20-25': 3, '25-30': 3, '30-35': 2, '35-40': 0, '40-45': 0, '45-50': 0, '50-55': 2, '55-60': 0, '60-65': 0, '65-70': 0, '70-75': 0, '75-80': 0, '80-85': 0, '85-90': 0, '90-95': 0,

In [14]:
##收集Reconnaissance的distribution dictionary
print("Collecting Reconnaissance distribution dictionary")

#Data preprocessing
Reconnaissance_df = train_dataset.loc[train_dataset['attack_cat'] == 7]
#Feature scaling
ReconnaissanceMatrix=scaler.fit_transform(Reconnaissance_df.values)
print(ReconnaissanceMatrix.shape)

Reconnaissance_Distribution_set = {}
for i in range(feature_num):
    StatisticObj = statistics(ReconnaissanceMatrix, i)
    Reconnaissance_Distribution_set[str(i)] = StatisticObj

print(Reconnaissance_Distribution_set)
save_obj(Reconnaissance_Distribution_set, "Reconnaissance_Distribution_set")

print("End of collecting Reconnaissance distribution dictionary")

(13987, 24)
{'0': {'0-5': 13658, '5-10': 115, '10-15': 31, '15-20': 17, '20-25': 12, '25-30': 14, '30-35': 3, '35-40': 11, '40-45': 4, '45-50': 7, '50-55': 12, '55-60': 4, '60-65': 8, '65-70': 2, '70-75': 5, '75-80': 7, '80-85': 0, '85-90': 2, '90-95': 6, '95-100': 69}, '1': {'0-5': 13982, '5-10': 1, '10-15': 0, '15-20': 0, '20-25': 0, '25-30': 0, '30-35': 0, '35-40': 0, '40-45': 0, '45-50': 0, '50-55': 2, '55-60': 0, '60-65': 0, '65-70': 0, '70-75': 0, '75-80': 0, '80-85': 0, '85-90': 0, '90-95': 0, '95-100': 2}, '2': {'0-5': 13911, '5-10': 49, '10-15': 12, '15-20': 2, '20-25': 2, '25-30': 0, '30-35': 0, '35-40': 0, '40-45': 0, '45-50': 0, '50-55': 0, '55-60': 0, '60-65': 0, '65-70': 0, '70-75': 0, '75-80': 0, '80-85': 0, '85-90': 0, '90-95': 0, '95-100': 0}, '3': {'0-5': 13982, '5-10': 1, '10-15': 0, '15-20': 0, '20-25': 0, '25-30': 0, '30-35': 0, '35-40': 0, '40-45': 0, '45-50': 0, '50-55': 2, '55-60': 0, '60-65': 0, '65-70': 0, '70-75': 0, '75-80': 0, '80-85': 0, '85-90': 0, '90-95

In [15]:
##收集Shellcode的distribution dictionary
print("Collecting Shellcode distribution dictionary")

#Data preprocessing
Shellcode_df = train_dataset.loc[train_dataset['attack_cat'] == 8]
#Feature scaling
ShellcodeMatrix=scaler.fit_transform(Shellcode_df.values)
print(ShellcodeMatrix.shape)

Shellcode_Distribution_set = {}
for i in range(feature_num):
    StatisticObj = statistics(ShellcodeMatrix, i)
    Shellcode_Distribution_set[str(i)] = StatisticObj

print(Shellcode_Distribution_set)
save_obj(Shellcode_Distribution_set, "Shellcode_Distribution_set")

print("End of collecting Shellcode distribution dictionary")

(1511, 24)
{'0': {'0-5': 1280, '5-10': 156, '10-15': 41, '15-20': 12, '20-25': 6, '25-30': 1, '30-35': 2, '35-40': 3, '40-45': 2, '45-50': 2, '50-55': 2, '55-60': 1, '60-65': 1, '65-70': 0, '70-75': 0, '75-80': 0, '80-85': 0, '85-90': 0, '90-95': 1, '95-100': 1}, '1': {'0-5': 761, '5-10': 0, '10-15': 0, '15-20': 0, '20-25': 1, '25-30': 1, '30-35': 0, '35-40': 0, '40-45': 0, '45-50': 2, '50-55': 2, '55-60': 0, '60-65': 0, '65-70': 0, '70-75': 508, '75-80': 508, '80-85': 0, '85-90': 0, '90-95': 0, '95-100': 239}, '2': {'0-5': 222, '5-10': 184, '10-15': 135, '15-20': 38, '20-25': 195, '25-30': 257, '30-35': 137, '35-40': 148, '40-45': 48, '45-50': 35, '50-55': 40, '55-60': 44, '60-65': 7, '65-70': 7, '70-75': 0, '75-80': 14, '80-85': 0, '85-90': 0, '90-95': 0, '95-100': 14}, '3': {'0-5': 761, '5-10': 0, '10-15': 0, '15-20': 0, '20-25': 0, '25-30': 1, '30-35': 0, '35-40': 0, '40-45': 0, '45-50': 0, '50-55': 2, '55-60': 0, '60-65': 0, '65-70': 0, '70-75': 0, '75-80': 508, '80-85': 0, '85-90

In [16]:
##收集Worms的distribution dictionary
print("Collecting Worms distribution dictionary")

#Data preprocessing
Worms_df = train_dataset.loc[train_dataset['attack_cat'] == 8]
#Feature scaling
WormsMatrix=scaler.fit_transform(Worms_df.values)
print(WormsMatrix.shape)

Worms_Distribution_set = {}
for i in range(feature_num):
    StatisticObj = statistics(WormsMatrix, i)
    Worms_Distribution_set[str(i)] = StatisticObj

print(Worms_Distribution_set)
save_obj(Worms_Distribution_set, "Worms_Distribution_set")

print("End of collecting Worms distribution dictionary")

(1511, 24)
{'0': {'0-5': 1280, '5-10': 156, '10-15': 41, '15-20': 12, '20-25': 6, '25-30': 1, '30-35': 2, '35-40': 3, '40-45': 2, '45-50': 2, '50-55': 2, '55-60': 1, '60-65': 1, '65-70': 0, '70-75': 0, '75-80': 0, '80-85': 0, '85-90': 0, '90-95': 1, '95-100': 1}, '1': {'0-5': 761, '5-10': 0, '10-15': 0, '15-20': 0, '20-25': 1, '25-30': 1, '30-35': 0, '35-40': 0, '40-45': 0, '45-50': 2, '50-55': 2, '55-60': 0, '60-65': 0, '65-70': 0, '70-75': 508, '75-80': 508, '80-85': 0, '85-90': 0, '90-95': 0, '95-100': 239}, '2': {'0-5': 222, '5-10': 184, '10-15': 135, '15-20': 38, '20-25': 195, '25-30': 257, '30-35': 137, '35-40': 148, '40-45': 48, '45-50': 35, '50-55': 40, '55-60': 44, '60-65': 7, '65-70': 7, '70-75': 0, '75-80': 14, '80-85': 0, '85-90': 0, '90-95': 0, '95-100': 14}, '3': {'0-5': 761, '5-10': 0, '10-15': 0, '15-20': 0, '20-25': 0, '25-30': 1, '30-35': 0, '35-40': 0, '40-45': 0, '45-50': 0, '50-55': 2, '55-60': 0, '60-65': 0, '65-70': 0, '70-75': 0, '75-80': 508, '80-85': 0, '85-90